In [31]:
# Dependencies and Setup
#import hvplot.pandas
import pandas as pd
import gmaps
import os
import requests

# Import API key
from api_keys import gmaps_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hilo,19.7297,-155.0900,27.38,87,100,5.66,US,1680121787
1,1,cumanayagua,22.1494,-80.2022,31.23,38,11,0.62,CU,1680121875
2,2,tuktoyaktuk,69.4541,-133.0374,-18.00,71,0,2.06,CA,1680121875
3,3,harper,4.3750,-7.7169,26.69,86,27,2.65,LR,1680121875
4,4,hobart,-42.8794,147.3294,12.92,74,75,9.77,AU,1680121793


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
#%%capture --no-display

# Configure the map plot
gmaps.configure(api_key = gmaps_key)

# Display the map
city_locations = city_data_df[["Latitude","Longitude"]].astype(float)
city_humidity = city_data_df["Humidity"].astype(float)
maximum_humidity = max(city_humidity)
fig= gmaps.figure()
humidity_heatmap = gmaps.heatmap_layer(city_locations,weights = city_humidity,max_intensity = maximum_humidity )
fig.add_layer(humidity_heatmap)
fig

Figure(layout=FigureLayout(height='420px'))

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Wind Speed"]<=5) & (city_data_df["Cloudiness"]==0) & \
                                 (city_data_df["Max Temp"]<=28)  & (city_data_df["Max Temp"]>=21)]
                            
# Drop any rows with null values
ideal_weather_df = ideal_weather.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,cape town,-33.9258,18.4232,22.99,57,0,2.57,ZA,1679939079
63,63,los llanos de aridane,28.6585,-17.9182,22.86,41,0,4.12,ES,1679939204
67,67,ewa beach,21.3156,-158.0072,24.97,83,0,2.06,US,1679939020
110,110,pisco,-13.7000,-76.2167,27.03,61,0,3.60,PE,1679939234
128,128,kapaa,22.0752,-159.3190,24.55,89,0,2.57,US,1679939167
150,150,arlit,18.7369,7.3853,27.19,9,0,2.94,NE,1679939276
154,154,chuy,-33.6971,-53.4616,23.74,49,0,2.77,UY,1679939279
296,296,rong kwang,18.3390,100.3174,26.71,34,0,0.76,TH,1679939368
397,397,catuday,16.2923,119.8062,25.63,78,0,1.55,PH,1679939448
436,436,mulchen,-37.7167,-72.2333,23.92,16,0,2.48,CL,1679939211


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.loc[:, [ "City","Country", "Latitude","Longitude","Humidity"]]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Latitude,Longitude,Humidity,Hotel Name
45,cape town,ZA,-33.9258,18.4232,57,
63,los llanos de aridane,ES,28.6585,-17.9182,41,
67,ewa beach,US,21.3156,-158.0072,83,
110,pisco,PE,-13.7000,-76.2167,61,
128,kapaa,US,22.0752,-159.3190,89,
150,arlit,NE,18.7369,7.3853,9,
154,chuy,UY,-33.6971,-53.4616,49,
296,rong kwang,TH,18.3390,100.3174,34,
397,catuday,PH,16.2923,119.8062,78,
436,mulchen,CL,-37.7167,-72.2333,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = { "type" : "hotel",
          "keyword" : "hotel",
          "radius" = radius,
          "key" = gmaps_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = # YOUR CODE HERE
    params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = # YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Hotel del Real
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
81,makakilo city,US,21.3469,-158.0858,81,Embassy Suites by Hilton Oahu Kapolei
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,No hotel found
211,laguna,US,38.4210,-121.4238,79,Holiday Inn Express & Suites
240,tikaitnagar,IN,26.9500,81.5833,59,No hotel found
265,san quintin,MX,30.4833,-115.9500,74,Jardines Hotel
340,santa rosalia,MX,27.3167,-112.2833,56,Hotel del Real


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)